# 0. Abstract and Outline

The purpose of this document is to introduce the central tradeoff in supervised learning theory: The bias-variance tradeoff. We introduce a unified generalization error framework for both regression and classification. The training-validation-testing triplet framework has been presented for model selection and generalization error estimation. We also discuss an alternative approach of cross-validation to fine-tune model parameters. We decompose the generalization error of a supervised learning model into bias, variance, and noise so as to crystalize the bias-variance tradeoff and the relationship between model complexity and prediction errors. Regularization techniques will also be introduced to address overfitting. The learning theory, modeling details, the case study, and the Python code are included in this document. 

Below is the outline of this document:

- In [Section 1](#section_1), we discuss supervised learning in a unified framework.

- In [Section 2](#section_2), we introduce the train-validate-test triplet and apply it to the Kwai user retention prediction case.

- In [Section 3](#section_3), we introduce $k-$fold cross validation and apply it to the Kwai user retention prediction case.

- In [Section 4](#section_4), we present the bias-variance tradeoff and introduce the regularization method (such as lasso and ridge regressions) to address overfitting. The boston housing prediction problem is also discussed as a demonstration for lasso regression.

<a id='section_1'></a>
# 1. Supervised Learning in a Unified Framework

Let us recall a general form of supervised learning. We have a sample data set, $\mathcal D:=\{Y_i,X_{ij}:1\le i\le n,1\le j \le p\}$, to train a model (a regressor or a classifier) $\hat f(\cdot)$. For a regression problem, the outcome variable $Y_i$ is continuous; for a classification problem, the outcome $Y_i$ is discrete.

The sample data $\mathcal D$ should come from a broader population and we hope that the sample could reasonably represent the entire population. For example, the sample data may come from a well-designed survey with which we could understand the whole population well. Mathematically, we are assuming that both the sample data $\mathcal D$ and the population data are the **SAME** probablistic data generating process/distribution. Therefore, we could extract information from the sample data to reason about the data generating process and the whole population. This rationale is called **generalization** in supervised learning.

For this document, let us consider the following supervised learning problem:

----------

<font color=red>
    
- **Input:** A sample training data set $\mathcal D:=\{Y_i,X_{ij}:1\le i\le n,1\le j \le p\}$. Here, the outcome variable $Y$ could be either discrete or continuous
- **Output:** A model fitted on $\mathcal D$ such that the **generalization error** $\mathbb E[\mathcal L(Y,\hat f(X))]$ is **minimized**, where $\mathcal L(\cdot,\cdot)$ is the loss/error function for the problem of interest and the expectation is taken with respect to the "true" distribution that generates the sample data and the population data. In most cases, the loss function $\mathcal L(Y,\hat f(X))$ takes the forms of $(Y-\hat f(X))^2$ (squared-loss), $|Y-\hat f(X)|$ (absolute value/$L^1$ loss), $1\{Y\ne \hat f(X)\}$ （0-1 loss）and $-Y_i\log(\hat g(X_i))-(1-Y_i)\log(1-\hat g(X_i))$ (cross-entropy). 

</font>
    
------------

As is clear from the above problem formulation, our goal is not to build a model that minimizes the prediction error on any specific data set (even not the validation or testing set). Instead, supervised learning seeks to minimize the prediction error on the entire population (even beyond the current population, please think about this from a probablistic perspective).

In the next two sections, we will introduce two systematic approaches to select the "best" model and evaluate the generalization error of the selected model. 

<a id='section_2'></a>
# 2. Train-Validate-Test Triplet

In supervised learning, a key problem is to effectively and efficiently select a prediction model with the smallest **generalization error** using a data set at hand $\mathcal D:=\{Y_i,X_{ij}:1\le i\le n,1\le j \le p\}$. The set of candidate models is denoted as $\mathcal F:=\{f_1(\cdot),f_2(\cdot),...,f_L(\cdot)\}$. The loss function is denoted as $\mathcal L(\cdot,\cdot)$. The core idea of the train-validate-test triplet approach is that using the data following the same distribution as the population but not overlapping with the data used to train the model could unbiasedly estimate the generalization error. We summarize the entire train-validate-test procedure as follows:

-------------

<font color=red>

- **Step 1. Data Split.** Randomly split the data $\mathcal D$ into 3 nonoverlapping subsets: $\mathcal D_{tr}$ for training (i.e., to fit a machine learning model), $\mathcal D_{va}$ for validation (to pick up the best model), and $\mathcal D_{te}$ for testing (to evaluate the selected model). Usually most of the data should be put to $\mathcal D_{tr}$ and a smaller portion should be allocated to $\mathcal D_{va}$ or $\mathcal D_{te}$.

- **Step 2. Training.** Using the training set $\mathcal D_{tr}$, train a model for each candidate in $\mathcal F$, which we denote as $\hat{\mathcal{F}}=\{\hat f_1,\hat f_2,...,\hat f_L\}$.

- **Step 3. Validation.** Use the validation set $\mathcal D_{va}$ to estimate the generalization error of each model $\hat f_i\in\hat{\mathcal{F}}$:

$$\mathbb E[\mathcal L(Y,\hat f_i(X))]\approx \hat e_i=\frac{1}{|\mathcal D_{va}|}\sum_{j\in \mathcal D_{va}} \mathcal L(Y_j,\hat f_i(X_j))$$
where $|\mathcal D_{va}|$ is the sample size of the validation data set $\mathcal D_{va}$.

Select the model with the smallest generalization error, which we denote as $\hat e_*=\min\{\hat e_1,\hat e_2,...,\hat e_L\}$, and the selected model is denoted as $\hat f_*(\cdot)$:

$$\hat f_*(\cdot)=\mbox{argmin}_{\hat f(\cdot)\in \hat{\mathcal{F}}}\frac{1}{|\mathcal D_{va}|}\sum_{j\in \mathcal D_{va}} \mathcal L(Y_j,\hat f(X_j))$$

- **Step 4. Testing.** Finally, use the testing data set $\mathcal D_{te}$ to evaluate the generalization error of the selected model $\hat f_*(\cdot)$:

$$\hat{err}_*=\frac{1}{|\mathcal D_{te}|}\sum_{j\in \mathcal D_{te}} \mathcal L(Y_j,\hat f_*(X_j))$$
where $|\mathcal D_{te}|$ is the sample size of the testing set $\mathcal D_{te}$.

Then, $\hat{err}_*$ is an unbiased estimation of the true generalization error for the "best" model in $\mathcal F$. 

</font>

----------------


One should also note that $\hat e_*$ is an **underestimate** of the generalization error. If one only cares about selecting the optimal model, but not the performance of the optimal model, we only need to split the data into two parts, training and validation so that the validation-error minimization model can be selected.

## 2.1. Train-Validate-Test in Action: the Kwai User Retention Case

We now revisit the user retention prediction problem of Kwai. Recall that we use the following short-term consumption behaviors of 28 days (from day-$(t-27)$ to day-$t$) to predict whether a user will be active on the platform between day-$(t+28)$ and day-$(t+34)$. :

- The user's total video watching time (in minutes) of the 28 days spent on the hot page (发现页);
- The user's total video watching time (in minutes) of the 28 days spent on the follow page (关注页);
- The user's total live watching time (in minutes) of the 28 days spent on the follow page (关注页);
- The user's total video watching time (in minutes) of the 28 days spent on the nearby page (同城页);
- The user's total live watching time (in minutes) of the 28 days spent on the nearby page (同城页);
- The user's total time (in minutes) of the 28 days spent on the comment area (评论区);
- The user's total number of comments of the 28 days posted on the comment area.

As a demonstration of the train-validate-test triplet, we examine two candidate models: 

- **Model1:** logistic regression model with all consumption variables as features;
- **Model2:** logistic regression model with only video consumptions as features. 

The performance metrics we will use in this demonstration is **AUC**.

To begin with, We first read the data set ``Kwai_Retention.csv`` and take a look at the first few rows of the data set.

In [1]:
import sys 
import numpy as np
import pandas as pd
import statsmodels as sm
import sklearn
import scipy as sp
%matplotlib inline 
import matplotlib.pyplot as plt
import math
import random

# Set the random seed such that the results are replicable.

random.seed(666)

In [2]:
kwai_retention = pd.read_csv("Kwai_Retention.csv")
kwai_retention.head()

,user_id,retention,h_video,f_video,f_live,n_video,n_live,comment_duration,comment_num
0,1,1,14.333127,82.440787,50.715540,25.448967,31.928108,24.547305,2
1,2,0,0.000000,0.000000,169.315973,0.000000,100.053897,0.873836,0
2,3,1,24.886263,0.000000,8.932622,55.413817,136.752137,9.372263,1
3,4,0,180.547294,67.597604,0.000000,9.487704,13.855340,23.045255,2
4,5,1,124.664241,81.037966,156.721750,37.109027,0.000000,26.401544,3


Next, we randomly split the data set into 3 parts, 60% into the training set, 20% into the validation set, 20% into the testing set.

In [3]:
from sklearn.model_selection import train_test_split

X = kwai_retention.drop(columns=['user_id','retention'])
y = kwai_retention['retention'].ravel()

# First randomly select 60% of the data into the training set.
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4)

# Then randomly select 50% of the rest of the data into testing set.
X_validate, X_test, y_validate, y_test = train_test_split(X_temp, y_temp, test_size=0.5)

In [4]:
print(X_train.shape)
print(X_test.shape)
print(X_validate.shape)

(60000, 7)
(20000, 7)
(20000, 7)


The next step is to train both models on the training set.

In [5]:
from sklearn.linear_model import LogisticRegression as LogitReg

LR_1 = LogitReg().fit(X_train,y_train)
LR_2 = LogitReg().fit(X_train[['h_video','f_video','n_video']],y_train)

Next, we compute the out-of-sample AUC of each model on the validation set.

In [6]:
from sklearn.metrics import  roc_curve, auc

# Compute the AUC for model 1.
fpr_1,tpr_1,thresholds_1 = roc_curve(y_validate, LR_1.predict_proba(X_validate)[:,1])
roc_auc_1 = auc(fpr_1,tpr_1)

# Compute the AUC for model 2.
fpr_2,tpr_2,thresholds_2 = roc_curve(y_validate, LR_2.predict_proba(X_validate[['h_video','f_video','n_video']])[:,1])
roc_auc_2 = auc(fpr_2,tpr_2)

In [7]:
print('The validation AUC of Model 1 is %0.4f.'% roc_auc_1)
print('The validation AUC of Model 2 is %0.4f.'% roc_auc_2)

The validation AUC of Model 1 is 0.7272.
The validation AUC of Model 2 is 0.6695.


As we can see, the validation AUC of Model 1 is slightly better than that of Model 2. Therefore, we select Model 1 and test its generalization AUC on the testing set.

In [8]:
fpr_te,tpr_te,thresholds_te = roc_curve(y_test, LR_1.predict_proba(X_test)[:,1])
roc_auc_te = auc(fpr_te,tpr_te)

print('The testing AUC of Model 1 is %0.4f.' % roc_auc_te)

The testing AUC of Model 1 is 0.7323.


As we can see, the testing AUC of Model 1 is 0.733, which is a unbiased estimate of the "true" generalization AUC of the model.

<a id='section_3'></a>
# 3. $K-$Fold Cross Validation

An alternative approach to systematically select a supervised learning model and evaluate its generalization error is $k-$fold cross validation ([Wiki Page](https://en.wikipedia.org/wiki/Cross-validation_(statistics))). One key drawback of the training-validation-testing triplet is that it requires a lot of data. Cross-validation, however, helps reduce the amount of data needed to select the optimal model and estimate the generalization error. 

The key idea of cross-validation is that we split the data into different parts, and use some parts to fit the model and some parts to validate the model. Furthermore, we repeat the above process the other way arround by switching the training and validationing sets. 

As with the training-validation-testing triplet, the data set at hand is denoted as $\mathcal D:=\{Y_i,X_{ij}:1\le i\le n,1\le j \le p\}$, the set of candidate models is denoted as $\mathcal F:=\{f_1(\cdot),f_2(\cdot),...,f_L(\cdot)\}$, and the loss function is denoted as $\mathcal L(\cdot,\cdot)$. The $k-$fold cross validation procedure is performed as follows:


-----
<font color=red>

- **Step 1. Data Split.** Randomly split the data $\mathcal D$ into 2 nonoverlapping subsets: $\mathcal D_{tr}$ for training (i.e., to fit and cross-validate a machine learning model), and $\mathcal D_{te}$ for testing (to evaluate the selected model). Usually most of the data should be put to $\mathcal D_{tr}$ and a smaller portion should be allocated to $\mathcal D_{te}$.

- **Step 2. $k-$Fold Training Data.** Randomly split the training data $\mathcal D_{tr}$ into $k$ parts of equal size, which we denote as $\mathcal D^1,\mathcal D^2,...,\mathcal D^k$.

- **Step 3. Cross-Validation.** For each model in $l\in\mathcal F$, train this model on $k-1$ parts of the training data $\{\mathcal D^1,\mathcal D^2,...,\mathcal D^{i-1},\mathcal D^{i+1},...,\mathcal D^k\}$, and estimate the generalization error on the remaining part $\mathcal D^i$, denoted as $\hat e_l^i$. The average generalization error of model $l$ is therefore

$$\hat e_l=\frac{1}{k}\sum_{i=1}^k\hat e_l^i$$

- **Step 4. Model Selection.** Select the model with the smallest average generalization error, which we denote as $\hat e_{i^*}=\min\{\hat e_1,\hat e_2,...,\hat e_L\}$. Retrain model $f_{i^*}(\cdot)$ on the entire training set $\mathcal D_{tr}$, which we obtain $\hat f^{cv}_*(\cdot)$.

- **Step 5. Testing.** Finally, use the testing data set $\mathcal D_{te}$ to evaluate the generalization error of the selected model $\hat f^{cv}_*(\cdot)$, which we denote as $\hat{err}^{cv}_*$.

Then, $\hat{err}^{cv}_*$ is an unbiased estimation of the true generalization error for the "best" model in $\mathcal F$. In theory, if the sample size of the training set $|\mathcal D_{tr}|$ is sufficiently large, the smallest average generalization error of cross validation, $\hat e_{i^*}$ will also converge to the true generalization error. 

</font>

------

An advantage of the cross-validation approach over the train-validate-test approach is that it has a smaller variances since the performance evaluation produced by cross-validation is averaged accross errors of different rounds. The number of iterations/folds for cross-validation, $k$, is usually set between 5 and 10. 

We illustrate the $k-$fold cross validation procedure as the following figure:

<img src="kfolds.png" width=800>

Next, we introduce a demostration to show how to implement cross validation in Python by fine-tuning the number-of-neighbor parameter $k$ in the $k-$NN algorithm. 

## 3.1.  Case Study: $k-$Fold Cross Validation for $k-$NN

We implement the $k-$Fold Cross Validation framework to select the best number of neighbors for $k-$NN. Specifically, we want to select the best $k-$NN model to predict the long-term user retention for Kwai.

To begin with, we read the data.

In [9]:
kwai_retention = pd.read_csv("Kwai_Retention.csv")
kwai_retention.head()

,user_id,retention,h_video,f_video,f_live,n_video,n_live,comment_duration,comment_num
0,1,1,14.333127,82.440787,50.715540,25.448967,31.928108,24.547305,2
1,2,0,0.000000,0.000000,169.315973,0.000000,100.053897,0.873836,0
2,3,1,24.886263,0.000000,8.932622,55.413817,136.752137,9.372263,1
3,4,0,180.547294,67.597604,0.000000,9.487704,13.855340,23.045255,2
4,5,1,124.664241,81.037966,156.721750,37.109027,0.000000,26.401544,3


Then, we randomly split the data into training and validation sets by randomly sample 70% of the data into the training set and the rest into the testing set.

In [10]:
X = kwai_retention.drop(columns=['retention','user_id'])
y = kwai_retention['retention'].ravel()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Recall that, for $k-$NN, we need to standardize the features by substracting their mean (of the training set) and devide by their standard deviation (of the training set).

In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train),columns = X_train.columns)
X_test = pd.DataFrame(scaler.fit_transform(X_test),columns = X_test.columns)

Then we load necessary packages for cross-validation and $k-$NN.

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

Then, we use 5-fold cross-validation to fine-tune the number-of-neighbor $k$ in the $k$NN model. The [documentation for GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).

In [13]:
# The candidate models/hypter-parameters: $k-$NN (k=1,2,...,10)

# The possible values of the hyper-parameter n_neighbors (k)
param_grid = [{'n_neighbors':range(1,11)}]

# The model is the kNN classifier
clf = KNeighborsClassifier()

# The evaluation method is AUC.
grid_1 = GridSearchCV(clf, param_grid, scoring = 'roc_auc',cv=5)

# Fit the model and perform k-fold cross-validation.
grid_1.fit(X_train,y_train)

print("The best k is ",grid_1.best_params_) 
print("\n") 
print("The AUC associated with the best k is %0.4f." % grid_1.best_score_)

The best k is  {'n_neighbors': 10}


The AUC associated with the best k is 0.6764.


From the cross-validation results, we know that the best $k$NN model is obtained when $k=10$. So we fit a 10-NN model on the entire training set. First, we make predictions on the testing set based on the $10-$NN model.

In [14]:
kNN = KNeighborsClassifier(n_neighbors = 10).fit(X_train,y_train)

fpr,tpr,thresholds = roc_curve(y_test, kNN.predict_proba(X_test)[:,1])
roc_auc = auc(fpr,tpr)

print('The testing AUC for the 10-NN model is %0.4f.' % roc_auc)

The testing AUC for the 10-NN model is 0.6765.


We also print the confusion matrix and calculate the accuracy of the predicted outcomes and evaluate the model accuracy. 

In [15]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, kNN.predict(X_test)).ravel()

In [16]:
def printCM(tn, fp, fn, tp):
    print('{: <9} {: <9} {: <9}'.format(' ',' ','predicted'))
    print('         --------------------')
    print('{: <9}|{: <9d}|{: <9d}'.format('Acutal',0,1))
    print('-----------------------------')
    print('{: <9d}|{: <9d}|{: <9d}'.format(0,tn,fp))
    print('-----------------------------')
    print('{: <9d}|{: <9d}|{: <9d}'.format(1,fn,tp))
printCM(tn, fp, fn, tp)

                    predicted
         --------------------
Acutal   |0        |1        
-----------------------------
0        |3757     |5361     
-----------------------------
1        |4024     |16858    


In [17]:
accuracy = (tn + tp) / (tn + fp + fn + tp)
print('The testing accuracy of 10-NN model is %0.4f' % accuracy)

The testing accuracy of 10-NN model is 0.6872


<a id='section_4'></a>
# 4. Bias-Variance Tradeoff

We are now ready to characterize a fundamentally (and notorious) issue in supervised learning: the Bias-Variance trade-off ([Wiki Page](https://en.wikipedia.org/wiki/Bias%E2%80%93variance_tradeoff)). This trade-off is so fundamental and prevalent that it offers some actionable insights on how we could reduce the total geeneralization error of a supervised model.

## 4.1. Bias-Variance Decomposition

We first present the decomposition of error into bias, variance, and noise terms, which highlights the intrinsic trade-off between bias and variance. We consider a data sample $\mathcal D=\{Y_i,X_{ij}:1\le i\le n,1\le j \le p\}\}$. For notational simplicity, we make a functional assumption that, for each $i$, $Y_i=f(X_i)+\epsilon_i$, where $f(\cdot)$ is a real-valued function and $\epsilon_i$ has mean zero ($\mathbb E[\epsilon_i]=0$). For a new data in the general population $(X,Y)$ also satisfies this functional form: $Y=f(X)+\epsilon$, where $X$ follows the same distribution as $X_i$ and $\epsilon$ follows from the same distribution as $\epsilon_i$ for each $i$. Based on the sample data $\mathcal D$, we train a model $\hat f(\cdot,\mathcal D)$, where we explicitly specify the dependence of the model on the training set $\mathcal D$. Our goal is to find $\hat f(X,\mathcal D)$ that approximates the true data generating function $f(X)$ as close as possible for all $X$ in the population, measured by a certain loss function $\mathcal L(\cdot,\cdot)$. In this section, we focus on the squared loss: $(Y-\hat f(X,\mathcal D))^2$ 

In fact, regardless of how we train the model $\hat f(\cdot,\mathcal D)$, we have the following decomposition of the squared loss for an unseen data $(X,Y)\notin \mathcal D$:

<font color=red>

$$\underbrace{\mathbb E_{\mathcal D}[(Y-\hat f(X,\mathcal D))^2]}_{\mbox{Squared Error}}=\underbrace{(\mathbb E_{\mathcal D}[\hat f(X,\mathcal D)]-f(X))^2}_{\mbox{Bias}} + \underbrace{\mathbb E_{\mathcal D}[(\mathbb E_{\mathcal D}[\hat f(X,\mathcal D)]-\hat f(X,\mathcal D))^2]}_{\mbox{Variance}}+\underbrace{Var (\epsilon)}_{\mbox{Noise}}$$

</font>

where $Var(\cdot)$ refers to taking variance, and the expectation $\mathbb E_{\mathcal D}(\cdot)$ is taken with respect to the training set $\mathcal D$ sampled from underlying "true" data generating process of $(X,Y)$. The above three terms have the following interpretations:


------------

<font color=red>

- **Bias** is the inherent error with your model caused by, e.g., simplifying assumptions, even with infinitely many training data. As is clear from its definition, the bias term measures how far away is the fitted model $\hat f(\cdot,\mathcal D)$ from the "true model" $f(\cdot)$ under the underlying data generating process. It essentially captures the degree of underfitting for the model trained on $\mathcal D$.

- **Variance** is the amount of the change the model will have if trained on a different data set. As is clear from its definition, the variance term measures how far away is the fitted outcome $\hat f(X,\mathcal D)$ from its mean under the true data generating process. It essentially captures the degree of overfitting for the model.

- **Noise** is the variance of the intrinsic error $\epsilon$, which is independent of the choice of model.
</font>

-----------------

A pictorial illustration of the bias and variance can be found in the following picture:

<img src="target.png" width=500>

From the bias-variance decomposition formula, we know that both bias and variance are sensitive to the model $\hat f(\cdot,\mathcal D)$. We have the following figure to demonstrate the trade-off between bias and variance under different model complexities.

<img src="Overfitting.png" width=750>



As shown by the figure above, a more complex model could have more flexibilities to fit the "true" model (also called the ground truth in the machine learning community) so that the bias will decrease in this case. Meanwhile, it may overfit the training data so that the variance of the model may be too high. On the other hand, a less complex model would not be that sensitive to changes in the training set (low variance), but may be too simple to capture the true model (high bias). In the following figure, we demonstrate this bias-variance tradeoff and show how total generalization error changes with model complexity. As we can see, the total generalization error is minimized when the model complexity is at a moderate sweet spot.

<img src="biasvariance.png" width=500>

Next subsection is devoted to a systematic way of addressing over-fitting: Regularization.

## 4.2. Regularization

Regularization ([Wiki Page](https://en.wikipedia.org/wiki/Regularization_(mathematics) )) refers to a systematic approach to address the overfitting problem of overly complex models by introducing a penalty term in the objective function that penalizes an overly complex model. Specifically, given a training data set $\mathcal D=\{Y_i,X_{ij}:1\le i\le n,1\le j \le p\}\}$, a family of models $\mathcal F$, and the loss function $\mathcal L(\cdot,\cdot)$, the unregularized training is to find a model $\hat f(\cdot)$ that minimizes the in-sample error.

<font color=red>

$$\hat f(\cdot)=\mbox{argmin}_{f(\cdot)\in\mathcal F}\frac{1}{n}\sum_{i=1}^n\mathcal L(Y_i,f(X_i))$$

</font>
    
Under regularization, we introduce a regularizer, or a penality function associated with the model $R(f)\ge0$. In particular, if $f$ is more complex, $R(f)$ will become larger. The model complexity is represented differently for different models. For a linear regression or logistic regression, a more complex model means the number of features $p$ is larger. For $k$-NN, a more complex model means $k$ is small. We will discuss what it means by mode complexity for other machine learning models as we discuss them.  

The regularized model fitting is performed as follows:


<font color=red>


$$\hat f(\cdot,\lambda)=\mbox{argmin}_{f(\cdot)\in\mathcal F}[\underbrace{\frac{1}{n}\sum_{i=1}^n\mathcal L(Y_i,f(X_i))}_{\mbox{Loss}}+\underbrace{\lambda R(f)}_{\mbox{Penalty}}]$$
    
</font>

where the hyper parameter $\lambda\ge0$. The regularization term $\lambda R(f)$ penalizes overly complex models. In particular, the parameter $\lambda$ trades off bias (under-fitting) and variance (over-fitting). Let us now introduce some commonly used regularized regressions:

- **Lasso linear regression**: 

<font color=red>

$$\hat\beta=\mbox{argmin}_{\beta}\underbrace{\frac{1}{n}\sum_{i=1}^n(Y_i-\beta_0-\sum_{j=1}^p\beta_jX_{ij})^2}_{\mbox{Loss}}+\underbrace{\lambda\sum_{j=1}^p|\beta_j|}_{\mbox{Penalty}}$$

</font>

In lasso regression, the loss is usually referred to as the mean square error and the penalty is called $L^1$ penalty.

- **Ridge linear regression**:

<font color=red>

$$\hat\beta=\mbox{argmin}_{\beta}\underbrace{\frac{1}{n}\sum_{i=1}^n(Y_i-\beta_0-\sum_{j=1}^p\beta_jX_{ij})^2}_{\mbox{Loss}}+\underbrace{\frac{\lambda}{2}\sum_{j=1}^p|\beta_j|^2}_{\mbox{Penalty}}$$

</font>

In ridge regression, the loss is also the mean square error and the penalty is called $L^2$ penalty.

- **Elastic net linear regression**: 

<font color=red>

$$\hat\beta=\mbox{argmin}_{\beta}\underbrace{\frac{1}{n}\sum_{i=1}^n(Y_i-\beta_0-\sum_{j=1}^p\beta_jX_{ij})^2}_{\mbox{Loss}}+\underbrace{\lambda\sum_{j=1}^p[\frac{1}{2}(1-\alpha)|\beta_j|^2+\alpha|\beta_j|]}_{\mbox{Penalty}}$$

</font>
Clearly, the elastic net linear regression is a combination of lasso ($\alpha=1$) and ridge ($\alpha=0$) regressions ([Wiki Page](https://en.wikipedia.org/wiki/Elastic_net_regularization)).

- **Lasso logistic regression**: We can also regularize logistic regression by substracting $L^1$ and $L^2$ penalties to the maximum likelihood estimation:

<font color=red>

$$\hat\beta=\mbox{argmax}_{\beta}\underbrace{\sum_{i=1}^n\left(Y_i\log\left(\frac{\exp(\beta_0+\sum_{j=1}^p\beta_jX_{ij})}{1+\exp(\beta_0+\sum_{j=1}^p\beta_jX_{ij})}\right)+(1-Y_i)\log\left(\frac{1}{1+\exp(\beta_0+\sum_{j=1}^p\beta_jX_{ij})}\right)\right)}_{\mbox{Log-Likelihood}}-\underbrace{\lambda\sum_{j=1}^p|\beta_j|}_{\mbox{Penalty}}$$

</font>
    
- **Ridge logistic regression**: 

<font color=red>

$$\hat\beta=\mbox{argmax}_{\beta}\underbrace{\sum_{i=1}^n\left(Y_i\log\left(\frac{\exp(\beta_0+\sum_{j=1}^p\beta_jX_{ij})}{1+\exp(\beta_0+\sum_{j=1}^p\beta_jX_{ij})}\right)+(1-Y_i)\log\left(\frac{1}{1+\exp(\beta_0+\sum_{j=1}^p\beta_jX_{ij})}\right)\right)}_{\mbox{Log-Likelihood}}-\underbrace{\frac{\lambda}{2}\sum_{j=1}^p|\beta_j|^2}_{\mbox{Penalty}}$$

</font>    
Finally, we briefly discuss the properties of lasso ($L^1$) and ridge ($L^2$) regularizations. From their formmulation, we know that lasso and ridge will shrink the estimated coefficients $\hat \beta$ to 0. Only the most explanatory features will be retained with regularized regressions. In particular, lasso typitcally yields much fewer non-zero entries of $\hat\beta$ than ridge or OLS. Thus, lasso shrinks $\hat\beta$ to 0 more sharply. For ridge regression, it shrinks $\hat\beta$'s to zero more softly. As is shown in the following figure (where $L1$ refers lasso regression and $L2$ refers to ridge regression), with $L2$ regularizer, the variable that minimizes the function $f(x)+\alpha|x|^2$ is closer to 0 than the minimizer of $f(x)=(2x-1)^2$. With $L1$ regularizer, the minimizer of $f(x)+\alpha |x|$ is directly shrinked to 0.  

<img src="L1L2.png" width=750>

We usually use the train-validate-test triplet or cross validation to fine tune the parameter $\lambda$.

## 4.3. Regularization Case Study: Boston Housing Price 

The Boston housing data set is a commonly used one in machine learning. This data set contains the following variables:

- **CRIM**: Per capita crime rate
- **ZN**: How much of the land is zoned for large residential properties
- **INDUS**: Proportion of the area used for industry
- **CHAS**: Binary variable, 1 if a census tract is next to the Charles River
- **NOX**: Concentration of nitrous oxides in the air, a measure of air pollution
- **RM**: Average number of rooms per dwelling
- **AGE**: Proportion of owner-occupied units built before 1940
- **DIS**: Measure of how far the tract is from centers of employment in Boston
- **RAD**: Measure of closeness to important highways
- **TAX**: Property tax per 10,000 dollars of value
- **PTRATIO**: Pupil to teacher ratio by town
- **TRACT**: Index of the tract
- **lstat**: Percentage values of lower status population. 
- **MEDV (outcome variable)**: Median value of owner-occupied homes, measured in thousands of dollars

Our job is to use cross-validation to fine tune the regularization parameter $\lambda$ in lasso regression. To begin with, we load the data into Python.

In [18]:
df_BH = pd.read_csv("BostonHousing.csv")
df_BH.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   crim     506 non-null    float64
 1   zn       506 non-null    float64
 2   indus    506 non-null    float64
 3   chas     506 non-null    int64  
 4   nox      506 non-null    float64
 5   rm       506 non-null    float64
 6   age      506 non-null    float64
 7   dis      506 non-null    float64
 8   rad      506 non-null    int64  
 9   tax      506 non-null    int64  
 10  ptratio  506 non-null    float64
 11  b        506 non-null    float64
 12  lstat    506 non-null    float64
 13  medv     506 non-null    float64
dtypes: float64(11), int64(3)
memory usage: 55.5 KB


We import the ```ElasticNet``` package. See [this documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html).

In [19]:
# Import the ElasticNet model
from sklearn.linear_model import ElasticNet

We randomly split the data into training and testing sets, the former with 70% of the data and the latter with 30%.

In [20]:
X =  df_BH.drop(columns=['medv'])
y = df_BH['medv'].ravel()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Then, we use cross-validation to fine tune the regularization parameter $\lambda$. We set the number of folds to be 5. In ```sklearn.linear_model.ElasticNet```, the parameter $\lambda$ is named as ```alpha```.

In [21]:
# Grid: 0.01, 0.02, ..., 0.5
param_grid = [{'alpha':np.arange(0.01,0.51,0.01)}]

# l1_ratio means Lasso
Lasso_LinReg = ElasticNet(l1_ratio = 1)
grid_1 = GridSearchCV(Lasso_LinReg, param_grid,cv=5)
grid_1.fit(X_train,y_train.ravel())
print("The best alpha parameter is",grid_1.best_params_) 
print('\n') 
print("The corresponding R-squared is %0.4f." % grid_1.best_score_) 

The best alpha parameter is {'alpha': 0.01}


The corresponding R-squared is 0.7480.


In [22]:
# Print the cross validation results.

grid_1.cv_results_

{'mean_fit_time': array([0.00174074, 0.00136299, 0.00132036, 0.00131645, 0.00131469,
        0.00131383, 0.00131521, 0.00131445, 0.00141616, 0.00134068,
        0.00132661, 0.00134397, 0.00130486, 0.00133963, 0.00145025,
        0.00132976, 0.00137353, 0.00133643, 0.00136294, 0.00135689,
        0.00135622, 0.00135598, 0.00136456, 0.00139623, 0.00138893,
        0.00135255, 0.00134411, 0.0013545 , 0.00135632, 0.0013526 ,
        0.00135303, 0.00135455, 0.00137258, 0.0013483 , 0.00134773,
        0.00137119, 0.00134234, 0.00135484, 0.00129952, 0.00133762,
        0.00136533, 0.00134764, 0.00134349, 0.0013526 , 0.00134273,
        0.00133948, 0.00134654, 0.00135527, 0.00132327, 0.00129304]),
 'std_fit_time': array([4.04272948e-04, 4.37505304e-05, 1.07611065e-05, 5.27244778e-06,
        7.54307554e-06, 7.42642301e-06, 7.84480459e-06, 6.06427393e-06,
        1.76910082e-04, 5.68325884e-05, 3.52789898e-05, 3.56730288e-05,
        4.57614636e-06, 4.87975681e-05, 1.52082007e-04, 1.94632522e-0

Recall that, for lasso regression, ``l1_ratio`` is set to 1. We also construct a grid search for $\lambda$ in the set ``np.arange(0.01,0.51,0.01)``, i.e., all $\lambda$s between 0.01 and 0.5, with step-size 0.01. As reported in the result above, the best $\lambda=0.01$. Next, we re-train the model on the entire training set.

In [23]:
Lasso_Reg = ElasticNet(l1_ratio = 1,alpha = 0.01).fit(X_train,y_train.ravel())

# Compute the testing R-squared.

print("The out-of-sample R-squared is %0.4f." % Lasso_Reg.score(X_test,y_test))

The out-of-sample R-squared is 0.6448.


# 5. Concluding Remarks

**Generalization** is the central notion in supervised learning, which refers to the procedure of optimizing the model performance on **unseen** data generated by the **true** data-generating process of the entire population. Train-Validate-Test and $k$-fold cross-validation are standard (and scientifically sound) procedures to select the best model that minimizes the generalization error. Cross-validation is more time-consuming, but more widely used especially when the data is scarce.

A fundamental trade-off for supervised learning is the one between **bias** and **variance**, which can be nicely quantified by an elegant decomposition of the generalization error. A **simple** model may not be able to capture the ground-truth and, therefore, suffers from a **high bias**. In contrast, a **complex** model may overfit the training data and, thus, suffers from a **high variance**. A good model represents a wise balance between over-fitting and under-fitting. In practice, the bias-variance trade-off is usually paramerized by some model hyper-parameters that guide the training of the model. Therefore, balancing the bias-variance trade-off is often operationalized through fine-tuning hyper-parameters through enumeration, i.e., grid-search.

Besides the grid search method introduced in this ``Jupyter Notebook``, another very commonly used approach to finetune hyper-parameters is the **randomized search** [(See this link for more details.)](https://www.kaggle.com/willkoehrsen/intro-to-model-tuning-grid-and-random-search) method. See also [this paper](https://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf) and the [documentation for randomized search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html). The key idea is that, when the number of parameters we need to fine tune is large, grid search may be too efficient. So we impose a distribution such that the hyper-parameters should follow, and each time we sample a hyper-parameter from the distribution to search for the optimal parameter. We will introduce the randomized search approach in more details later in this course.